In [1]:
#!pip install h2o

In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

import pylab as plt
import seaborn as sns

%matplotlib inline

pd.set_option('display.max_columns', None)
from sklearn.metrics import mean_squared_error as mse

In [3]:
rbnb = pd.read_csv('../data/train.csv')
rbnb_test = pd.read_csv('../data/test.csv')

In [4]:
# Modifico la columna 'bathrooms_text' para extraer de la string el número de baños

def extract_bathrooms(x):
    try:
        return re.findall('\d+', x)[0]
    except:
        return 0.5

rbnb.bathrooms_text = rbnb.bathrooms_text.apply(extract_bathrooms)
rbnb.bathrooms_text = rbnb.bathrooms_text.astype(float)


# Modifico la columna 'amenities' para quitarle caracteres.

rbnb.amenities = rbnb.amenities.apply(lambda x: x.replace('"', '').replace('[', '').replace(']', ''))
rbnb.amenities = rbnb.amenities.apply(lambda x: x.count(','))

In [5]:
# Modifico la columna 'bathrooms_text' para extraer de la string el número de baños

def extract_bathrooms(x):
    try:
        return re.findall('\d+', x)[0]
    except:
        return 0.5

rbnb_test.bathrooms_text = rbnb_test.bathrooms_text.apply(extract_bathrooms)
rbnb_test.bathrooms_text = rbnb_test.bathrooms_text.astype(float)


# Modifico la columna 'amenities' para quitarle caracteres.

rbnb_test.amenities = rbnb_test.amenities.apply(lambda x: x.replace('"', '').replace('[', '').replace(']', ''))
rbnb_test.amenities = rbnb_test.amenities.apply(lambda x: x.count(','))

# H2O

## Entreno el modelo

In [6]:
'''
from sklearn.model_selection import train_test_split as tts


train, test = tts(rbnb, test_size=.2, random_state=42)

train.shape, test.shape'''

'\nfrom sklearn.model_selection import train_test_split as tts\n\n\ntrain, test = tts(rbnb, test_size=.2, random_state=42)\n\ntrain.shape, test.shape'

In [7]:
import h2o
from h2o.automl import H2OAutoML

In [8]:
# inicializamos el modelo h2o

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,49 mins 31 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,25 days
H2O_cluster_name:,H2O_from_python_usuario_e7m3js
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.705 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [9]:
h2train=h2o.H2OFrame(rbnb)
h2test=h2o.H2OFrame(rbnb_test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [10]:
X=[c for c in h2train.columns if c!='price']

y='price'

In [11]:
# inicia auto-machine-learning

automl=H2OAutoML(max_models=500,
                 seed=42,   # random_state
                 max_runtime_secs=10000,
                 sort_metric='RMSE')

In [ ]:
# entrena

automl.train(x=X,
             y=y,
             training_frame=h2train)

AutoML progress: |█
20:38:06.930: AutoML: XGBoost is not available; skipping it.
20:38:06.933: _train param, Dropping bad and constant columns: [picture_url, neighbourhood_group_cleansed, calendar_updated, listing_url, name, description, bathrooms_text, bathrooms, scrape_id]

███
20:38:15.902: _train param, Dropping bad and constant columns: [picture_url, neighbourhood_group_cleansed, calendar_updated, listing_url, name, description, bathrooms_text, bathrooms, scrape_id]
20:38:21.405: _train param, Dropping bad and constant columns: [picture_url, neighbourhood_group_cleansed, calendar_updated, listing_url, name, description, bathrooms_text, bathrooms, scrape_id]

█████
20:39:29.502: _train param, Dropping bad and constant columns: [picture_url, neighbourhood_group_cleansed, calendar_updated, listing_url, name, description, bathrooms_text, bathrooms, scrape_id]

█
20:39:45.134: _train param, Dropping bad and constant columns: [picture_url, neighbourhood_group_cleansed, calendar_updated,

In [ ]:
print('[INFO] Leader board:')

leader_board=automl.leaderboard

leader_board.head()

In [ ]:
H20_1 = automl.leader

In [ ]:
# prediciones del lider

y_pred=automl.leader.predict(h2test)

In [ ]:
# Para testear la precisión de mi modelo

'''from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred_test)'''

## Prediccion del test real para hacer el summit

In [ ]:
# Predicción del test con el modelo leader

y_pred_test = automl.leader.predict(h2test)

In [ ]:
submission = h2o.as_list(y_pred_test)

In [ ]:
rbnb_test_y = rbnb_test.id

In [ ]:
submission['id'] = rbnb_test_y

In [ ]:
submission = submission.rename(columns={'predict': 'price'})

In [ ]:
submission.to_csv('../data/H2O_max.csv', sep=',', index=False)

In [ ]:
y_pred = h2o.as_list(y_pred)



In [ ]:
mse(test.price, y_pred.predict, squared=False)